In [18]:
!.\myenv\Scripts\pip install numpy scikit-learn xgboost imblearn mlflow --upgrade mlflow


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\k.hegde\Desktop\learnings\Learning_MLflow\myenv\Scripts\python.exe -m pip install --upgrade pip


In [2]:

import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [3]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [5]:
#logistic regression

log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



In [6]:
#random forest
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



In [7]:
#xgboost

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



In [8]:
# Handle class imbalance using SMOTETomek and then Train XGBoost

from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [9]:

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



In [10]:
models = [
    (
        "Logistic regression",
        {"C":1,"solver":"liblinear"},
        LogisticRegression(),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "Random Forest",
        {"n_estimators":30,"max_depth":3},
        RandomForestClassifier(),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder":False,"eval_metric":"logloss"},
        XGBClassifier(),
        (X_train,y_train),
        (X_test,y_test)
    ),
    (
        "XGBClassifier with SMOTE",
        {"use_label_encoder":False,"eval_metric":"logloss"},
        XGBClassifier(),
        (X_train_res,y_train_res),
        (X_test,y_test)
    )
    
]

In [11]:
reports = []

for model_name, params,model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [12]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [17]:
# Initialize MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("AnomalyDetection")


for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
        
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })       
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/10/29 16:34:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/29 16:34:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/29 16:34:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic regression at: http://localhost:5000/#/experiments/412791393524591808/runs/65fcbf81912f4ce8bcddfe39319dd9d2
🧪 View experiment at: http://localhost:5000/#/experiments/412791393524591808


2025/10/29 16:34:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/29 16:34:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/412791393524591808/runs/c7d2462aeec74c2385d7e348e5bd847f
🧪 View experiment at: http://localhost:5000/#/experiments/412791393524591808


2025/10/29 16:34:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/10/29 16:34:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/412791393524591808/runs/f96cea1b95ad4b8582aa77938c4d1eda
🧪 View experiment at: http://localhost:5000/#/experiments/412791393524591808


2025/10/29 16:34:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier with SMOTE at: http://localhost:5000/#/experiments/412791393524591808/runs/a5e30b08b36f4e1586caaa44dceb0b9b
🧪 View experiment at: http://localhost:5000/#/experiments/412791393524591808


In [ ]:
result = mlflow.register_model(
    
    "runs:/a5e30b08b36f4e1586caaa44dceb0b9b/","XGBClassifier with SMOTE"
)

In [21]:
mlflow.register_model(model_uri="runs:/a5e30b08b36f4e1586caaa44dceb0b9b/model", name="XGB-with-smote")

Registered model 'XGB-with-smote' already exists. Creating a new version of this model...
2025/10/29 17:00:57 WARNING mlflow.tracking._model_registry.fluent: Run with id a5e30b08b36f4e1586caaa44dceb0b9b has no artifacts at artifact path 'model', registering model based on models:/m-4fc9ca62f8db47ceb65093ad3bcbdcb0 instead
2025/10/29 17:00:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-with-smote, version 2
Created version '2' of model 'XGB-with-smote'.


<ModelVersion: aliases=[], creation_timestamp=1761737457336, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761737457336, metrics=None, model_id=None, name='XGB-with-smote', params=None, run_id='a5e30b08b36f4e1586caaa44dceb0b9b', run_link='', source='models:/m-4fc9ca62f8db47ceb65093ad3bcbdcb0', status='READY', status_message=None, tags={}, user_id='', version='2'>

### load the model

In [28]:
model_name = "XGB-with-Smote" 
model_version = 1
model_uri = f"models:/{model_name}@challenger"

loaded_model = mlflow.xgboost.load_model(model_uri="runs:/a5e30b08b36f4e1586caaa44dceb0b9b/model")
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [31]:
dev_model_uri = f"models:/{model_name}@challenger"
prod_model = 'anomaly_detection_prod'

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri= dev_model_uri, dst_name=prod_model)

Successfully registered model 'anomaly_detection_prod'.
Copied version '2' of model 'XGB-with-smote' to version '1' of model 'anomaly_detection_prod'.


<ModelVersion: aliases=[], creation_timestamp=1761741302341, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761741302341, metrics=None, model_id=None, name='anomaly_detection_prod', params=None, run_id='a5e30b08b36f4e1586caaa44dceb0b9b', run_link='', source='models:/XGB-with-smote/2', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [32]:
model_uri = f"models:/{prod_model}@champion"

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])